In [1]:
from sys import path
import os
import glob
path.append('/home/rui/blacksid/mew-p')
DATA_PATH = '/home/rui/blacksid/data/dockfuture'
import numpy as np
import pytz
from datetime import datetime
from datetime import timedelta
from datetime import time
import cProfile
import matplotlib.pyplot as plt
from mewp.simulate.wrapper import TrendAlgoWrapper
from mewp.simulate.runner import SingleRunner
import pandas as pd
from mewp.util.clock import Clock
from mewp.util.candle import Candle
from mewp.util.candle import CandleCreator
from mewp.util.candle import CandleAnalysis

In [2]:
os.chdir('/home/rui/blacksid/data/stock/daily')

In [33]:
product = 'J'

In [34]:
contract_file_list = glob.glob("{}1*.csv".format(product))
CONTRACT_LIST = [x.split('.')[0] for x in contract_file_list]
CONTRACT_DF = {}
for contract in CONTRACT_LIST:
    temp = pd.read_csv(contract + '.csv')
    CONTRACT_DF[contract] = temp

In [21]:
dates = [str(x).split(' ')[0] for x in pd.date_range('2010-01-01','2016-06-01').tolist()]

In [6]:
def get_contract_info_at_day(contract, date):
    df = CONTRACT_DF[contract]
    index = (df.datetime == date)
    if sum(index) == 0:
        return np.nan
    elif sum(index) == 1:
        return df[index]
    else:
        print '********warning: more than two rows for a day'
        return np.nan

In [7]:
def is_trade_day(date):
    for contract in CONTRACT_LIST:
        if not type(get_contract_info_at_day(contract, date))==float:
            return True
    return False

In [8]:
dates = [str(x).split(' ')[0] for x in pd.date_range('2010-01-01','2016-08-16').tolist()]
ALL_DATES = []
for date in dates:
    if is_trade_day(date):
        ALL_DATES.append(date)

In [7]:
# Brief: input a date, output the main contract of that date.
def get_contract_score(date):
    date_contract_list = []
    contract_volume_list = []
    contract_position_list = []
    for contract in CONTRACT_LIST:
        df = get_contract_info_at_day(contract, date)
        if not type(df)==float:
            date_contract_list.append(contract)
            contract_volume_list.append(float(df.TotalVolumeTraded))
            contract_position_list.append(float(df.OpenInterest))
    if len(date_contract_list) == 0:
        return
    max_volume = max(contract_volume_list)
    max_position = max(contract_position_list)
    contract_volume_list = map(lambda x: x/max_volume, contract_volume_list)
    contract_position_list = map(lambda x: x/max_position, contract_position_list)
    relative_score_list = [(i+j)/2 for i,j in zip(contract_volume_list, contract_position_list)]
    score = dict(zip(date_contract_list, relative_score_list))
    return score

In [8]:
def get_main_contract(date_list, window):
    trade_day_list = []
    main_contract_list = []
    score_queue = []
    for date in date_list:
        contracts = None
        score = get_contract_score(date)
        if type(score) != dict:
            continue
        score_queue.append(score)
        if len(score_queue) > window:
            score_queue.pop(0)
        i = 0
        for score in reversed(score_queue):
            if contracts == None:
                contracts = set(score.keys())
            elif len(contracts.intersection(score.keys())) > 0:
                contracts = contracts.intersection(score.keys())
            else:
                break
            i+=1
        ma_score = {}
        for contract in contracts:
            ma = 0
            for k in range(1,i+1):
                ma+=score_queue[-k][contract]
            ma_score[contract] = ma
        main_contract = max(ma_score, key=ma_score.get)
        main_contract_list.append(main_contract)
        trade_day_list.append(date)
    return zip(trade_day_list, main_contract_list)

In [9]:
def last_trade_day_at_20_55(date_time):
    if date_time.weekday() == 0:
        last_friday = (date_time.date() + timedelta(days=4, weeks=-1))
        return datetime.combine(last_friday,time(20,55))
    else:
        yesterday = (date_time.date() - timedelta(days=1))
        return datetime.combine(yesterday, time(20,55))

In [10]:
def in_today_trade_time(today_datetime, date_time):
    start = last_trade_day_at_20_55(today_datetime)
    end = datetime.combine(today_datetime.date(), time(15,55))
    if date_time > start and date_time < end:
        return True
    else:
        return False

In [11]:
def get_close_price(contract, today):
    if contract[-3:] != 'csv':
        contract += '.csv'
    df1 = pd.read_csv(contract,index_col =0)
    bool_index = [in_today_trade_time(today, datetime.strptime(d, '%Y-%m-%d %H:%M:%S')) for d in df1.index]
    df1 = df1[bool_index]
    return df1.ClosingPx[-1]

In [12]:
def save_adjusted_continuous_price(date_list, window):
    os.chdir('/home/rui/blacksid/data/stock/daily')
    main_contract_list = get_main_contract(date_list, window)
    os.chdir('/home/rui/blacksid/data/5min')
    pre_main_contract = None
    pre_df = None
    price_adjust = 0
    for date, contract in reversed(main_contract_list):
        today = datetime.strptime(date,'%Y-%m-%d')
        if pre_main_contract==None:
            pass
        elif pre_main_contract == contract:
            pass
        else:
            price_adjust += float(get_close_price(pre_main_contract,today))\
                - float(get_close_price(contract, today))
        temp_df = pd.read_csv(contract+'.csv',index_col =0)
        bool_index = [in_today_trade_time(today, datetime.strptime(d, '%Y-%m-%d %H:%M:%S')) for d in temp_df.index]
        temp_df = temp_df[bool_index]
        temp_df['adjust'] = price_adjust
        pre_df = pd.concat([temp_df,pre_df])
        pre_main_contract = contract
    return pre_df
        

In [35]:
result = get_main_contract(dates,10)

In [36]:
result[-200:]

[('2015-08-06', 'J1509'),
 ('2015-08-07', 'J1509'),
 ('2015-08-10', 'J1509'),
 ('2015-08-11', 'J1509'),
 ('2015-08-12', 'J1509'),
 ('2015-08-13', 'J1601'),
 ('2015-08-14', 'J1601'),
 ('2015-08-17', 'J1601'),
 ('2015-08-18', 'J1601'),
 ('2015-08-19', 'J1601'),
 ('2015-08-20', 'J1601'),
 ('2015-08-21', 'J1601'),
 ('2015-08-24', 'J1601'),
 ('2015-08-25', 'J1601'),
 ('2015-08-26', 'J1601'),
 ('2015-08-27', 'J1601'),
 ('2015-08-28', 'J1601'),
 ('2015-08-31', 'J1601'),
 ('2015-09-01', 'J1601'),
 ('2015-09-02', 'J1601'),
 ('2015-09-07', 'J1601'),
 ('2015-09-08', 'J1601'),
 ('2015-09-09', 'J1601'),
 ('2015-09-10', 'J1601'),
 ('2015-09-11', 'J1601'),
 ('2015-09-14', 'J1601'),
 ('2015-09-15', 'J1601'),
 ('2015-09-16', 'J1601'),
 ('2015-09-17', 'J1601'),
 ('2015-09-18', 'J1601'),
 ('2015-09-21', 'J1601'),
 ('2015-09-22', 'J1601'),
 ('2015-09-23', 'J1601'),
 ('2015-09-24', 'J1601'),
 ('2015-09-25', 'J1601'),
 ('2015-09-28', 'J1601'),
 ('2015-09-29', 'J1601'),
 ('2015-09-30', 'J1601'),
 ('2015-10-0

In [15]:
df = save_adjusted_continuous_price(ALL_DATES, 10)

In [16]:
df

,BasisSpread,ClosingPx,HighPx,LimitDown,LimitUp,LowPx,OpenInterest,OpeningPx,TotalTurnover,TotalVolumeTraded,TradingDate,adjust
datetime,,,,,,,,,,,,
2010-01-04 09:04:00,NaN,7336.0,7354.0,NaN,NaN,7298.0,347042.0,7300.0,3.507271e+09,47882.0,2010-01-04,2340.0
2010-01-04 09:09:00,NaN,7300.0,7336.0,NaN,NaN,7288.0,342684.0,7336.0,2.713921e+09,37120.0,2010-01-04,2340.0
2010-01-04 09:14:00,NaN,7312.0,7316.0,NaN,NaN,7288.0,341170.0,7300.0,1.402278e+09,19194.0,2010-01-04,2340.0
2010-01-04 09:19:00,NaN,7314.0,7314.0,NaN,NaN,7300.0,338806.0,7312.0,9.086035e+08,12434.0,2010-01-04,2340.0
2010-01-04 09:24:00,NaN,7304.0,7320.0,NaN,NaN,7300.0,338476.0,7314.0,1.140836e+09,15600.0,2010-01-04,2340.0
2010-01-04 09:29:00,NaN,7306.0,7308.0,NaN,NaN,7294.0,336206.0,7304.0,1.449631e+09,19854.0,2010-01-04,2340.0
2010-01-04 09:34:00,NaN,7292.0,7308.0,NaN,NaN,7276.0,332058.0,7306.0,1.621959e+09,22248.0,2010-01-04,2340.0
2010-01-04 09:39:00,NaN,7288.0,7296.0,NaN,NaN,7282.0,331746.0,7292.0,1.277238e+09,17522.0,2010-01-04,2340.0
2010-01-04 09:44:00,NaN,7300.0,7300.0,NaN,NaN,7286.0,330048.0,7288.0,8.966037e+08,12294.0,2010-01-04,2340.0


In [17]:
df.to_csv('/home/rui/blacksid/data/adjusted_5min/{}0.csv'.format(product))

In [18]:
#result

In [19]:
# date_list = ALL_DATES
# window = 10
# os.chdir('/home/rui/blacksid/data/stock/daily')
# main_contract_list = get_main_contract(date_list, window)
# print main_contract_list
# os.chdir('/home/rui/blacksid/data/5min')
# pre_main_contract = None
# pre_df = None
# price_adjust = 0
# for date, contract in reversed(main_contract_list):
#     today = datetime.strptime(date,'%Y-%m-%d')
#     if pre_main_contract==None:
#         pass
#     elif pre_main_contract == contract:
#         pass
#     else:
#         print today
#         price_adjust += float(get_close_price(pre_main_contract,today))\
#                 - float(get_close_price(contract, today))
#     temp_df = pd.read_csv(contract+'.csv',index_col =0)
#     bool_index = [in_today_trade_time(today, datetime.strptime(d, '%Y-%m-%d %H:%M:%S')) for d in temp_df.index]
#     temp_df = temp_df[bool_index]
#     temp_df['adjust'] = price_adjust
#     pre_df = pd.concat([temp_df,pre_df])
#     pre_main_contract = contract

In [20]:
contract

'P1210'